<a href="https://colab.research.google.com/github/arbouria/Notas-Aprendizaje-y-Comportamiento-Adaptable-I/blob/main/doc/notebooks/interactive_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Redes Neuronales online demo

This is an interactive demo of a GDDM with leaky integration and exponentially collapsing bounds.

This demo can be run like a normal Jupyter notebook.  If you've never used Jupyter notebooks before, hover over both headings below ("Install PyDDM on Google Colab" and "Define the model and run the GUI") and press the play button on each.  An interactive demo will show below.  To make changes to the model and try out your changes, click on "Show code" and edit it.  When you are done, click on the play button again to update the demo.

In [ ]:
#@title Install PyDDM on Google Colab
!pip -q install git+https://github.com/mwshinn/PyDDM

In [ ]:
#@title Define the model and run the GUI
import pyddm
import pyddm.plot
import numpy as np
model = pyddm.gddm(drift=lambda x,leak,driftrate : driftrate - x*leak,
                   noise=1,
                   bound=lambda t,initial_B,collapse_rate : initial_B * np.exp(-collapse_rate*t),
                   starting_position="x0",
                   parameters={"leak": (0, 2),
                               "driftrate": (-3, 3),
                               "initial_B": (.5, 1.5),
                               "collapse_rate": (0, 10),
                               "x0": (-.9, .9)})

pyddm.plot.model_gui_jupyter(model)
# pyddm.plot.model_gui(model) # If not using a Jupyter notebook or Google Colab

# Ilustraciones de Modelos Neuronales Simples en Python para Colab

Estos ejemplos ilustran los conceptos básicos de los modelos neuronales vistos en las presentaciones, usando Python y NumPy. Puedes ejecutar cada celda de código de forma independiente.

**Modelos:**
1.  Perceptron (Umbral Lógico) para la función OR
2.  Neurona Simple con Sigmoide y Descenso de Gradiente para OR
3.  Red Multi-Capa (MLP) para la función XOR

## 1. Perceptron (Umbral Lógico) para OR

Este código implementa la idea básica de una neurona con una activación de umbral (sí/no) y la regla de aprendizaje del Perceptron. Intenta encontrar una línea que separe los casos (0,0) de los demás.

In [2]:
# Celda para el Perceptron OR

import numpy as np

# Función de activación de umbral (Step function)
def step_function(x):
  """Devuelve 1 si x es >= 0, de lo contrario 0."""
  return 1 if x >= 0 else 0

# --- Datos para la función OR ---
# Entradas (X): Cada fila es un ejemplo [x1, x2]
inputs = np.array([[0, 0],
                   [0, 1],
                   [1, 0],
                   [1, 1]])
# Salidas deseadas (Targets, T):
targets = np.array([0, 1, 1, 1])

# --- Inicialización del Perceptron ---
# Pesos (W): Uno por cada entrada. Inicializamos pequeños valores aleatorios.
# np.random.seed(42) # Descomentar para resultados reproducibles
weights = np.random.rand(2) * 0.1 # Dos pesos para dos entradas
# Sesgo (Bias, b): Un solo valor.
bias = np.random.rand(1) * 0.1
# Tasa de aprendizaje (learning rate, alpha): Qué tan grandes son los ajustes.
learning_rate = 0.1
# Número de épocas (pasadas completas por los datos)
epochs = 50

print("--- Perceptron para OR (Umbral Lógico) ---")
print(f"Pesos iniciales: {weights}, Sesgo inicial: {bias}")

# --- Entrenamiento ---
print("\nIniciando entrenamiento...")
converged = False
for epoch in range(epochs):
  total_error = 0
  for i in range(len(inputs)):
    # 1. Calcular la entrada neta (producto punto + sesgo)
    net_input = np.dot(inputs[i], weights) + bias

    # 2. Obtener la predicción usando la función de activación
    prediction = step_function(net_input)

    # 3. Calcular el error (Target - Prediction)
    error = targets[i] - prediction
    total_error += abs(error) # Acumulamos el error absoluto para ver progreso

    # 4. Actualizar pesos y sesgo (Regla de aprendizaje del Perceptron)
    weights += learning_rate * error * inputs[i]
    bias += learning_rate * error

  # Imprimir error cada N épocas para ver el progreso
  if (epoch + 1) % 10 == 0:
      print(f"Época {epoch + 1}/{epochs}, Error total en la época: {total_error}")
  # Detener si no hay error
  if total_error == 0 and not converged:
      print(f"Convergencia alcanzada en la época {epoch + 1}")
      converged = True
      # break # Podemos detener o dejar que corra hasta el final

if not converged:
    print("Puede que no haya convergido completamente en las épocas dadas.")

print(f"\nPesos finales: {weights}, Sesgo final: {bias}")

# --- Prueba después del entrenamiento ---
print("\nPredicciones finales:")
for i in range(len(inputs)):
  net_input = np.dot(inputs[i], weights) + bias
  prediction = step_function(net_input)
  print(f"Entrada: {inputs[i]}, Predicción: {prediction}, Esperado: {targets[i]}")


--- Perceptron para OR (Umbral Lógico) ---
Pesos iniciales: [0.04613017 0.01186871], Sesgo inicial: [0.06341379]

Iniciando entrenamiento...
Convergencia alcanzada en la época 3
Época 10/50, Error total en la época: 0
Época 20/50, Error total en la época: 0
Época 30/50, Error total en la época: 0
Época 40/50, Error total en la época: 0
Época 50/50, Error total en la época: 0

Pesos finales: [0.04613017 0.11186871], Sesgo final: [-0.03658621]

Predicciones finales:
Entrada: [0 0], Predicción: 0, Esperado: 0
Entrada: [0 1], Predicción: 1, Esperado: 1
Entrada: [1 0], Predicción: 1, Esperado: 1
Entrada: [1 1], Predicción: 1, Esperado: 1


## 2. Neurona Simple con Sigmoide y Descenso de Gradiente para OR

Este modelo usa la función sigmoide (salida continua entre 0 y 1) y ajusta los pesos usando Descenso de Gradiente para minimizar el error cuadrático. La salida puede interpretarse como una probabilidad.

In [3]:
# Celda para Neurona Sigmoide OR

import numpy as np

# Función de activación Sigmoide
def sigmoid(x):
  """Calcula la función sigmoide."""
  # Clip para evitar overflow/underflow en np.exp
  x_clipped = np.clip(x, -500, 500)
  return 1 / (1 + np.exp(-x_clipped))

# Derivada de la función sigmoide (necesaria para gradiente)
def sigmoid_derivative(x):
  """Calcula la derivada de la sigmoide."""
  sig = sigmoid(x)
  return sig * (1 - sig)

# --- Datos para la función OR ---
inputs = np.array([[0, 0],
                   [0, 1],
                   [1, 0],
                   [1, 1]])
targets = np.array([[0], [1], [1], [1]]) # Usamos [[],[]] para consistencia dimensional

# --- Inicialización ---
# np.random.seed(42) # Descomentar para resultados reproducibles
weights = np.random.rand(2, 1) * 0.1 # Forma (2, 1) para 2 entradas, 1 salida
bias = np.random.rand(1) * 0.1
learning_rate = 0.5 # Puede necesitar ajuste
epochs = 10000 # Generalmente necesita más épocas que el Perceptron simple

print("\n--- Neurona Sigmoide para OR (Descenso Gradiente) ---")
print(f"Pesos iniciales:\n{weights}\nSesgo inicial: {bias}")

# --- Entrenamiento ---
print("\nIniciando entrenamiento...")
for epoch in range(epochs):
  # Propagación hacia adelante (Forward Pass) - Todos los ejemplos a la vez
  net_input = np.dot(inputs, weights) + bias
  predictions = sigmoid(net_input)

  # Cálculo del error
  error = targets - predictions

  # Cálculo del gradiente (Regla Delta Generalizada)
  # Gradiente = error * derivada_activacion * entrada_transpuesta
  delta = error * sigmoid_derivative(net_input)

  # Ajuste de pesos y sesgo
  weights += learning_rate * np.dot(inputs.T, delta) # inputs.T alinea dimensiones
  bias += learning_rate * np.sum(delta, axis=0) # Sumamos ajustes para todas las muestras

  # Imprimir error (ECM) cada N épocas
  if (epoch + 1) % 1000 == 0:
    loss = np.mean(np.square(error)) # Calculamos el Error Cuadrático Medio
    print(f"Época {epoch + 1}/{epochs}, Error Cuadrático Medio: {loss:.6f}")

print(f"\nPesos finales:\n{weights}\nSesgo final: {bias}")

# --- Prueba después del entrenamiento ---
print("\nPredicciones finales:")
# Hacemos una pasada final con los pesos entrenados
net_input = np.dot(inputs, weights) + bias
predictions = sigmoid(net_input)
for i in range(len(inputs)):
  # Redondeamos para ver la clasificación binaria
  pred_class = 1 if predictions[i][0] >= 0.5 else 0
  print(f"Entrada: {inputs[i]}, Predicción: {predictions[i][0]:.4f} (Clase: {pred_class}), Esperado: {targets[i][0]}")



--- Neurona Sigmoide para OR (Descenso Gradiente) ---
Pesos iniciales:
[[0.07863678]
 [0.00149292]]
Sesgo inicial: [0.06227541]

Iniciando entrenamiento...
Época 1000/10000, Error Cuadrático Medio: 0.002906
Época 2000/10000, Error Cuadrático Medio: 0.001349
Época 3000/10000, Error Cuadrático Medio: 0.000869
Época 4000/10000, Error Cuadrático Medio: 0.000639
Época 5000/10000, Error Cuadrático Medio: 0.000504
Época 6000/10000, Error Cuadrático Medio: 0.000416
Época 7000/10000, Error Cuadrático Medio: 0.000354
Época 8000/10000, Error Cuadrático Medio: 0.000307
Época 9000/10000, Error Cuadrático Medio: 0.000272
Época 10000/10000, Error Cuadrático Medio: 0.000244

Pesos finales:
[[7.94183944]
 [7.94183914]]
Sesgo final: [-3.73488481]

Predicciones finales:
Entrada: [0 0], Predicción: 0.0233 (Clase: 0), Esperado: 0
Entrada: [0 1], Predicción: 0.9853 (Clase: 1), Esperado: 1
Entrada: [1 0], Predicción: 0.9853 (Clase: 1), Esperado: 1
Entrada: [1 1], Predicción: 1.0000 (Clase: 1), Esperado: 1


## 3. Red Multi-Capa (MLP) para XOR

El problema XOR no es linealmente separable, por lo que requiere una red con al menos una capa oculta. Este código implementa una MLP simple y usa retropropagación (Backpropagation) para ajustar los pesos de todas las capas.

In [4]:
import numpy as np

# Funciones sigmoide y su derivada (igual que antes)
def sigmoid(x):
  return 1 / (1 + np.exp(-np.clip(x, -500, 500))) # Añadido clip para estabilidad

def sigmoid_derivative(x):
  sig = sigmoid(x)
  return sig * (1 - sig)

# --- Datos para la función XOR ---
inputs = np.array([[0, 0],
                   [0, 1],
                   [1, 0],
                   [1, 1]])
targets = np.array([[0], [1], [1], [0]])

# --- Arquitectura de la Red ---
input_neurons = 2
hidden_neurons = 3 # Podemos probar con 2 o 3 neuronas ocultas
output_neurons = 1

# --- Inicialización ---
# np.random.seed(42)
# Pesos y sesgos para la capa oculta
weights_ih = np.random.rand(input_neurons, hidden_neurons) * 0.1
bias_h = np.random.rand(1, hidden_neurons) * 0.1
# Pesos y sesgos para la capa de salida
weights_ho = np.random.rand(hidden_neurons, output_neurons) * 0.1
bias_o = np.random.rand(1, output_neurons) * 0.1

learning_rate = 0.1 # XOR puede ser sensible a la tasa
epochs = 30000

print("\n--- Red Multi-Capa (MLP) para XOR ---")
print(f"Arquitectura: {input_neurons} (Entrada) -> {hidden_neurons} (Oculta) -> {output_neurons} (Salida)")

# --- Entrenamiento con Backpropagation ---
for epoch in range(epochs):
  total_loss = 0
  # Usaremos un bucle explícito para cada ejemplo para ver los pasos de backprop
  final_predictions = [] # Para guardar predicciones de la época

  for i in range(len(inputs)):
    input_layer = inputs[i:i+1] # Tomamos una fila (ejemplo) a la vez
    target_output = targets[i:i+1]

    # --- 1. Propagación hacia adelante (Forward Pass) ---
    # Capa Oculta
    hidden_layer_input = np.dot(input_layer, weights_ih) + bias_h
    hidden_layer_output = sigmoid(hidden_layer_input)

    # Capa de Salida
    output_layer_input = np.dot(hidden_layer_output, weights_ho) + bias_o
    predicted_output = sigmoid(output_layer_input)
    final_predictions.append(predicted_output[0][0])

    # --- 2. Cálculo del Error (en la salida) ---
    output_error = target_output - predicted_output
    total_loss += np.mean(np.square(output_error)) # Acumulamos ECM

    # --- 3. Retropropagación del Error (Backward Pass) ---
    # Gradiente para la capa de salida
    d_predicted_output = output_error * sigmoid_derivative(output_layer_input)

    # Error contribuido por la capa oculta (propagado hacia atrás)
    hidden_layer_error = np.dot(d_predicted_output, weights_ho.T)

    # Gradiente para la capa oculta
    d_hidden_layer = hidden_layer_error * sigmoid_derivative(hidden_layer_input)

    # --- 4. Actualización de Pesos y Sesgos ---
    # Capa de Salida
    weights_ho += learning_rate * np.dot(hidden_layer_output.T, d_predicted_output)
    bias_o += learning_rate * np.sum(d_predicted_output, axis=0, keepdims=True)

    # Capa Oculta
    weights_ih += learning_rate * np.dot(input_layer.T, d_hidden_layer)
    bias_h += learning_rate * np.sum(d_hidden_layer, axis=0, keepdims=True)

  # Imprimir error promedio cada N épocas
  if (epoch + 1) % 2000 == 0:
    avg_loss = total_loss / len(inputs)
    print(f"Época {epoch + 1}/{epochs}, Error Cuadrático Medio: {avg_loss:.6f}")


print("\n--- Prueba final MLP para XOR ---")
# Hacemos una pasada final con todos los inputs para mostrar resultados
hidden_layer_input = np.dot(inputs, weights_ih) + bias_h
hidden_layer_output = sigmoid(hidden_layer_input)
output_layer_input = np.dot(hidden_layer_output, weights_ho) + bias_o
final_predictions_all = sigmoid(output_layer_input)

for i in range(len(inputs)):
    pred_class = 1 if final_predictions_all[i][0] >= 0.5 else 0
    print(f"Entrada: {inputs[i]}, Predicción: {final_predictions_all[i][0]:.4f} (Clase: {pred_class}), Esperado: {targets[i][0]}")



--- Red Multi-Capa (MLP) para XOR ---
Arquitectura: 2 (Entrada) -> 3 (Oculta) -> 1 (Salida)
Época 2000/30000, Error Cuadrático Medio: 0.252699
Época 4000/30000, Error Cuadrático Medio: 0.252553
Época 6000/30000, Error Cuadrático Medio: 0.252430
Época 8000/30000, Error Cuadrático Medio: 0.252326
Época 10000/30000, Error Cuadrático Medio: 0.252230
Época 12000/30000, Error Cuadrático Medio: 0.252116
Época 14000/30000, Error Cuadrático Medio: 0.251845
Época 16000/30000, Error Cuadrático Medio: 0.248412
Época 18000/30000, Error Cuadrático Medio: 0.187177
Época 20000/30000, Error Cuadrático Medio: 0.049949
Época 22000/30000, Error Cuadrático Medio: 0.009068
Época 24000/30000, Error Cuadrático Medio: 0.004347
Época 26000/30000, Error Cuadrático Medio: 0.002775
Época 28000/30000, Error Cuadrático Medio: 0.002014
Época 30000/30000, Error Cuadrático Medio: 0.001571

--- Prueba final MLP para XOR ---
Entrada: [0 0], Predicción: 0.0341 (Clase: 0), Esperado: 0
Entrada: [0 1], Predicción: 0.9623 (C